In [76]:
import mido as md
import numpy as np
from collections import defaultdict
from sklearn import preprocessing
import heapq
import heapq

In [90]:
def generate_transition_probs(filepath):
    poops = []
    song = md.MidiFile(filepath)
    
    for track in song.tracks:
        poop = np.zeros((127, 127))
        prevnote = None
        n = np.zeros(127)
        for msg in track:
            x = vars(msg)
            if x['type'] == 'note_on' and x['velocity'] != 0:
                note = x['note']
                if prevnote is not None:
                    poop[prevnote][note] += 1
                    n[prevnote] += 1
                prevnote = note
        x = vars(track[0])
        if x['type'] == 'note_on' and x['velocity'] != 0:
                note = x['note']
                if prevnote is not None:
                    poop[prevnote][note] += 1
                    n[prevnote] += 1
                prevnote = note
        

        for i in range(127):
            if (n[i] != 0):
                poop[i] /= n[i]
        poops.append(poop)
    return poops


In [91]:
def transpose(filepath, offset):
    result = md.MidiFile()
    
    song = md.MidiFile(filepath)
    for track in song.tracks:
        newtrack = md.MidiTrack()
        result.tracks.append(newtrack)
        for msg in track:
            x = vars(msg)
            if x['type'] == 'note_on':
                note = x['note']
                note += offset
                newmessage = md.Message('note_on', note=note, velocity=x['velocity'], time=x['time'])
                newtrack.append(newmessage)
            else:
                newtrack.append(msg)
    result.save("midis/1.mid")

In [92]:
def findKeySignature(song):
    for i, track in enumerate(song.tracks):
        for msg in track:
            if msg.type == 'key_signature':
                return msg.key
            
def findTransposeOffset(filepath):
    song = md.MidiFile(filepath)
    keys = ['C', 'C#/Db', 'D', 'D#/Eb', 'E', 'F', 'F#/Gb', 'G', 'G#/Ab', 'A', 'A#/Bb', 'B']
    keySig = findKeySignature(song)
    print(keySig)
    for i in range(len(keys)):
        if keySig in keys[i]:
            offset = i
            break
    return offset

def removeZeroNotes(matrix):
    deleteHeap = []
    for j in range(len(matrix)):
        if not matrix[j].any(): #this means it's all zero
            if not matrix[:, j].any():
                heapq.heappush(deleteHeap, -j)
            
    while deleteHeap:
        deleted = -heapq.heappop(deleteHeap)
        np.delete(matrix, deleted, 0)
        np.delete(matrix, deleted, 1)
    return matrix
    
    
def findStationaryDist(matrix):
    try:
        w, v = np.linalg.eig(matrix.T)
    except LinAlgError as e:
        return None
    if 1 in w:
        return v[list(w).index(1)]
    else:
        return None


# dicking around and trying to make the left hand/right hand stuff better

In [104]:
song = md.MidiFile('./midis/bach846.mid')
song2_path = './midis/bach_847.mid'
song3_path = './midis/bach_850.mid'

In [105]:
song2 = transpose(song2_path, 3)
song3 = transpose(song3_path, -2)

songs = [song, song2, song3]

absoluteList = defaultdict(list)

for song in songs:
    for i, track in enumerate(song.tracks):
        for msg in track:
            if msg.type == 'note_on' and msg.velocity:
                absoluteList[i].append(msg)

    
    ####THIS IS BOUND TO FUCK UP AND HAVE MULTIPLE POTENTIAL LEFT/RIGHT HAND TRACKS, OR EVEN EMPTY TRACKS
    ####I MANUALLY INDEXED FOR NOW BELOW


absoluteTime = 0
rightHandTrack = []
leftHandTrack = []
for msg in absoluteList[1]:
    rightHandTrack.append((absoluteTime, msg.note, 1))
    absoluteTime += msg.time
    
absoluteTime = 0  
for msg in absoluteList[2]:
    leftHandTrack.append((absoluteTime, msg.note, 2))
    absoluteTime += msg.time

combined = rightHandTrack + leftHandTrack

AttributeError: 'NoneType' object has no attribute 'tracks'

In [108]:
#now, calculate transition probabilities for R->L. 'Combined' has tuples of (time, note, hand)
combined = sorted(combined, key=lambda x: x[0])
pMatrix = np.zeros((256, 256))
prevNote = None
for note in combined:
    if prevNote:
#         print("here")
        pMatrix[prevNote][note[1]+(128*(note[2]-1))] += 1
    prevNote = note[1]+(128*(note[2]-1))

pMatrix = preprocessing.normalize(pMatrix, axis=1, norm='l1')
    
#now, given this pMatrix, need to 'step through' it - HOWEVER
#for the coordinate generated, needs to be placed in a separate track

songLength = 500
starting_note = 66

from mido import Message, MidiFile, MidiTrack

delta = 240
current_note = starting_note

outfile = MidiFile()
trackRight = MidiTrack()
trackLeft = MidiTrack()
outfile.tracks.append(trackRight)
outfile.tracks.append(trackLeft)
trackRight.append(Message('program_change', program=12))
trackLeft.append(Message('program_change', program=12))

right = True
for i in range(songLength):
    if right:
        track = trackRight
    else:
        track = trackLeft
    
    track.append(Message('note_on', note = current_note%128, velocity = 60, time = delta))
    track.append(Message('note_on', note = current_note%128, velocity = 0, time = 80))
    possibleNexts = {i:pMatrix[current_note][i] for i in range(256) if pMatrix[current_note][i] > 0}
    notes = list(possibleNexts.keys())
    probabilities = [possibleNexts[n] for n in notes]
    prev_note = current_note
    current_note = np.random.choice(notes, 1, p=probabilities)[0]
    if current_note//128 != prev_note//128:
        right = not right
        

trackLeft.append(Message('program_change', program=12))

outfile.save('./testBachTotal.mid')

In [107]:
# noZeroes = removeZeroNotes(pMatrix)
# for i in noZeroes:
#     print(i)

In [97]:
song = md.MidiFile('./midis/chopinblack.mid')
trackNotes = defaultdict(list)
pMatrices = defaultdict(list)
for i, track in enumerate(song.tracks):
    print(i)
    pMatrix = np.zeros((128, 128))
    prevNote = None
    for msg in track:
        if msg.type == 'note_on' and msg.velocity:
#             print(msg)
            if prevNote:
                pMatrix[prevNote][msg.note] += 1
            prevNote = msg.note
    if pMatrix.any():
        pMatrices[i] = preprocessing.normalize(pMatrix, axis =1, norm = 'l1')


0
1
2
3
4
5
6


In [98]:
import numpy.linalg as LA

In [99]:
w, v = LA.eig(pMatrix)

In [100]:
w

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [101]:
v

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [89]:
for i in countMatrix:
    print(i)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0. 

[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   7.   0.   3.   0.   0.   7.   0.
  14.   0.   0.  18.   0.   5.   0.   7.   0.   0.  10.   0.   8.   0.   0.
   3.   0.  29.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.  

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   2.   0.   0.   0.   0.
   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   1.   0.   0.   0.   0.   5.   1.   0.   0.   0.   0.   0.   1.   0.   0.
   0.   0.   2.   0.   0.   0.   4.   1.   0.  10.   0.   2.   0.   0.  12.
   0.   0.  

  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0. 